In [174]:
import numpy as np
import tensorflow as tf

# 数据标准化

In [2]:
train_data = np.load(r'../Task3/result/train_data.npy')
train_label = np.load(r'../Task3/result/train_label.npy')

In [7]:
import resnet_v2 as res

In [143]:
name = np.unique(train_label)

y_num = [i for i in map(lambda x:np.where(x == name)[0][0],train_label)]

In [125]:
def get_batch(data,label,batch_size):
    i = 0
    while (i + batch_size) <= len(data):
        yield data[i:i+batch_size],label[i:i+batch_size]
        i += batch_size
    yield data[i:],label[i:]
    return None

In [182]:
G = tf.Graph()
with G.as_default():
    def get_input():
        x = tf.placeholder(tf.uint8,shape = [None,256*256])
        y = tf.placeholder(tf.uint8,shape = [None])
        return x,y

    def pre_process(x,y):
        x = tf.reshape(x,[-1,256,256,1])
        x_norm = tf.cast(x,tf.float32)
        y_norm = tf.one_hot(y,depth = 10)
        y_norm = tf.reshape(y_norm,[-1,10])
        return x_norm,y_norm
    x,y = get_input()
    x_norm,y_norm = pre_process(x,y)
    resnet = res.resnet_v2_50(x_norm,num_classes = 10)
    loss = tf.losses.softmax_cross_entropy(logits = resnet[0],onehot_labels = y_norm)
    tf.summary.scalar('Total_loss',loss)
    y_pred = tf.argmax(resnet[0])
    train_op = tf.train.AdamOptimizer().minimize(loss)

In [186]:
epoch = 1
batch_size = 16
writer = tf.summary.FileWriter(r'log/', G)

In [177]:
train_data = train_data[0:1000]
y_num = y_num[0:1000]

In [188]:
with tf.Session(graph = G) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epoch):
        data_gen = get_batch(train_data,y_num,batch_size)
        for j_num,j in enumerate(data_gen):
            data,label = j[0],j[1]
            _,show_loss = sess.run([train_op,loss],feed_dict = {x:data,y:label})
            if (j_num%10 == 0):
                print(f'epoch:{epoch},step:{j_num},loss:{show_loss}')
    saver = tf.train.Saver()
    saver.save(sess,r'./model/model.ckpt')

epoch:1,step:0,loss:2.525630235671997
epoch:1,step:10,loss:1.7436599731445312
epoch:1,step:20,loss:0.5652934312820435
epoch:1,step:30,loss:0.9011170864105225
epoch:1,step:40,loss:0.30647513270378113
epoch:1,step:50,loss:0.2624063789844513
epoch:1,step:60,loss:0.10510207712650299
